# Photoreceptor segmentation using [napari](https://napari.org/) and [cellpose](https://cellpose-napari.readthedocs.io/en/latest/index.html)

### Basic usage:
```python
viewer = napari.Viewer()
nbscreenshot(viewer)
viewer.close()
```

example notebooks [here](https://github.com/sofroniewn/napari-training-course/blob/master/lessons/)

In [ ]:
# !mamba install -c conda-forge devbio-napari --yes
# !jupyter labextension list

In [1]:
# image analysis
import napari
from magicgui import magicgui
from enum import Enum
import cellpose_napari
import cellpose
from cellpose import models
from napari.utils import nbscreenshot
from tifffile import imread
import napari_nikon_nd2
from skimage.io import imread
from magicgui import magicgui
# python basics and plotting
import numpy as np
import pandas as pd
import math
from scipy import ndimage
from scipy.stats import kruskal
from scipy.stats import zscore
from scipy.stats import sem
import scikit_posthocs
import scipy.spatial
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import svgutils
from svgutils.compose import *
# system interaction
import os.path
import time, sys, warnings, re
from tqdm.notebook import tqdm, trange
import importlib
import ipywidgets
from IPython.display import display
# lab brews
from plotParams import *
from lloyd import *


def getFileList(dirData,gene,fileNameMatch,addDetails=''):
   fileList = sorted([f for f in os.listdir(dirData) if not f.startswith('.')]) # get all the files in the data directory (excluding system files) and sort them alphabatically
   fileList = ([f for f in fileList if gene in f]) # select files that match the gene of interest
   fileList = ([f for f in fileList if any([s in f for s in fileNameMatch])]) # only keep relevant files
   if (addDetails=='path'):
      fileList = (['# filePath = \'' + f for f in fileList]) # add text before each file
      fileList = list(map(lambda st: str.replace(st, '.nd2', '\'; gene = \'' + gene + '\'; '), fileList)) #remove file extension (.nd2) and add text after each file
   elif (addDetails=='list'):
      fileList = (['\'' + f for f in fileList]) # add text before each file
      fileList = list(map(lambda st: str.replace(st, '.nd2', '\','), fileList)) #remove file extension (.nd2) and add text after each file
   else:
      fileList = list(map(lambda st: str.replace(st, '.nd2', ''), fileList)) #remove file extension (.nd2) and add text after each file
   return fileList


pBGMenu = ipywidgets.Dropdown(options=['Light', 'Dark'],value='Dark',disabled=False, layout=ipywidgets.Layout(width='20%', height='40px'))
pBGLabel = ipywidgets.widgets.Label('Select plotting style', layout=ipywidgets.Layout(width='20%', height='30px'))
pBG = ipywidgets.widgets.VBox([pBGLabel,pBGMenu])
pBG

In [2]:
applyPlotStyle(pBGMenu.value)
viewer = napari.Viewer()

Plotting style is Dark


In [ ]:
# viewer = napari.Viewer()

In [ ]:
viewer.close()

***
# Index <a id='Index'></a>
***
- [define directories](#dirDef)
- [Extraction of z planes for analysis](#zExtract)
- [Segmentation with cellpose](#cellSeg)
- [Manual correction of segmentation](#manualCuration)
- [Create thumbnails _WIP_](#thumbnails)
- [Quantification](#quantification)

***
## Define directories<a id='dirDefdirDef'>∮</a>
***
[Back to Index](#Index)

In [4]:
# define where all the imaging data is stored
dirData = '/Users/juanangueyra/Library/CloudStorage/GoogleDrive-angueyra@umd.edu/Shared drives/vldImaging/Data/20230209_lhx1aF0_m2G_s2C/'
# define where all the analysis files should be stored
# subdirectories for each gene and transgenic lines need to be manually made first
dirAnalysis = "/Users/juanangueyra/Library/CloudStorage/GoogleDrive-angueyra@umd.edu/Shared drives/vldImaging/Analysis/F0lhx1a/m2Gs2C/"

In [ ]:
gene = 'wt' # usually 'wt' or gene of interest (e.g. 'tbx2a', 'lhx1a')
gene = 'lhx1a' # usually 'wt' or gene of interest (e.g. 'tbx2a', 'lhx1a')
fileNameMatch = ['002','004'] # characters in filenames that define zoom-in files

# get a list with all the relevant files
print(*getFileList(dirData,gene,fileNameMatch,addDetails='path'), sep = "\n") #display the file list for copy and paste below

***
## Extract layers from z-stacks<a id='zExtract'>∮</a>
> Open z-stack &rarr; Pick limits in z for Maximum-Intensity Projection (_MIP_) &rarr; save MIPs &rarr; Ensure image was located in central retina
***
[Back to Index](#Index)

In [ ]:
# clear viewer if needed
viewer.layers.select_all()
viewer.layers.remove_selected()

In [ ]:
# clear viewer before any loading
viewer.layers.select_all()
viewer.layers.remove_selected()

# open files one by one (by commenting and uncommenting each one)

# wild-type files

# mutant files
# filePath = '20230209_5dpf_m2G_s2C_lhx1aF0_L01_002'; gene = 'lhx1a'; 
# filePath = '20230209_5dpf_m2G_s2C_lhx1aF0_L01_004'; gene = 'lhx1a'; 
# filePath = '20230209_5dpf_m2G_s2C_lhx1aF0_L02_002'; gene = 'lhx1a'; 
# filePath = '20230209_5dpf_m2G_s2C_lhx1aF0_L02_004'; gene = 'lhx1a'; 
# filePath = '20230209_5dpf_m2G_s2C_lhx1aF0_L03_002'; gene = 'lhx1a'; 
# filePath = '20230209_5dpf_m2G_s2C_lhx1aF0_L04_002'; gene = 'lhx1a'; 
# filePath = '20230209_5dpf_m2G_s2C_lhx1aF0_L04_004'; gene = 'lhx1a'; 
# filePath = '20230209_5dpf_m2G_s2C_lhx1aF0_L05_002'; gene = 'lhx1a'; 
# filePath = '20230209_5dpf_m2G_s2C_lhx1aF0_L05_004'; gene = 'lhx1a'; 
# filePath = '20230209_5dpf_m2G_s2C_lhx1aF0_L06_002'; gene = 'lhx1a'; 
# filePath = '20230209_5dpf_m2G_s2C_lhx1aF0_L06_004'; gene = 'lhx1a'; 
# filePath = '20230209_5dpf_m2G_s2C_lhx1aF0_L07_002'; gene = 'lhx1a'; 
# filePath = '20230209_5dpf_m2G_s2C_lhx1aF0_L07_004'; gene = 'lhx1a'; 
# filePath = '20230209_5dpf_m2G_s2C_lhx1aF0_L08_002'; gene = 'lhx1a'; 
# filePath = '20230209_5dpf_m2G_s2C_lhx1aF0_L08_004'; gene = 'lhx1a'; 
# filePath = '20230209_5dpf_m2G_s2C_lhx1aF0_L09_002'; gene = 'lhx1a'; 
# filePath = '20230209_5dpf_m2G_s2C_lhx1aF0_L09_004'; gene = 'lhx1a'; 
# filePath = '20230209_5dpf_m2G_s2C_lhx1aF0_L10_002'; gene = 'lhx1a'; 
# filePath = '20230209_5dpf_m2G_s2C_lhx1aF0_L10_004'; gene = 'lhx1a'; 
# filePath = '20230209_5dpf_m2G_s2C_lhx1aF0_L11_002'; gene = 'lhx1a'; 
# filePath = '20230209_5dpf_m2G_s2C_lhx1aF0_L11_004'; gene = 'lhx1a'; 
# filePath = '20230209_5dpf_m2G_s2C_lhx1aF0_L12_002'; gene = 'lhx1a'; 
# filePath = '20230209_5dpf_m2G_s2C_lhx1aF0_L12_004'; gene = 'lhx1a'; 
# filePath = '20230209_5dpf_m2G_s2C_lhx1aF0_L13_002'; gene = 'lhx1a'; 
filePath = '20230209_5dpf_m2G_s2C_lhx1aF0_L13_004'; gene = 'lhx1a'; 


layerNames = ['M','S']

# create subdirectory for analysis output
dirOut = dirAnalysis + gene + '/' + filePath + '/'
if (os.path.isdir(dirOut)==False):
   os.mkdir(dirOut)
   print('Created output directory')

print('Canvas cleared. Loading data...\n\n')

# load whole stack to figure out best z-planes for cellpose
viewer.open((dirData+filePath + '.nd2'), plugin='napari-nikon-nd2')
viewer.layers.remove(viewer.layers[len(viewer.layers)-1]) # remove transmitted detector image
viewer.layers.select_next();
if len(viewer.layers)==2: #no DAPI
   nChannels = 2
   viewer.layers[0].colormap = 'green'
   viewer.layers[0].name = layerNames[0]
   viewer.layers[1].colormap = 'magenta'
   viewer.layers[1].name = layerNames[1]
elif len(viewer.layers)==3: #DAPI
   nChannels = 3
   viewer.layers[0].colormap = 'gray'
   viewer.layers[0].name = 'N'
   viewer.layers[0].opacity = 0.33
   viewer.layers[1].colormap = 'green'
   viewer.layers[1].name = layerNames[0]
   viewer.layers[2].colormap = 'magenta'
   viewer.layers[2].name = layerNames[1]
   layerNames.append('N')

print('Images loaded!')

In [ ]:
# make mips (and remove any previous ones) by defining the z-limits to use
zlims = [14,16];
if len(viewer.layers)>nChannels:
    for l in viewer.layers[nChannels:]:
      viewer.layers.remove(l)

for l in viewer.layers[0:nChannels]:
    l.visible = False
#     viewer.layers.remove(l.name + '_mip')
    viewer.add_image(l.data[zlims[0]:zlims[1]].max(axis=0), blending='additive', colormap = l.colormap, name = l.name + "_mip")

In [ ]:
# SAVE in folder for batch cellpose analysis (when mips look good)
for name in layerNames:
   l = viewer.layers[name + '_mip']; l.save(dirOut + l.name + '.tiff')
print('MIP layers saved for ' + filePath)

### Establish landmarks in whole-eye MIP
- Use eyeMarks layer to mark:
    - [ ] Landmark #1: Center of FOV (where zoom in is centered)
    - [ ] Landmark #2: Center of Optic Nerve Head 
    - [ ] Landmark #3: Dorsal edge of eye
    - [ ] Landmark #4: Ventral edge of eye
    - [ ] Landmark #5: Temporal edge of eye (strike zone side)
    - [ ] Landmark #6: Nasal edge of eye
- Decide if stack should be excluded (and stop analysis here and make note in csv summary file)

In [ ]:
nMIP = len(layerNames)
for name in layerNames:
   lName = name + "_mip"
   viewer.layers[lName].scale = [1.5/8,1.5/8];
   viewer.layers[lName].translate = [0,+(512-1024*1.5/(8*2))]

# Load whole-eye MIP to establish landmarks
print('Loading eye mip...')
if str.endswith(filePath,'002'):
    mipPath = filePath[:-1] + '1_mip';
elif str.endswith(filePath,'004'):
    mipPath = filePath[:-1] + '3_mip';
elif str.endswith(filePath,'b'):
    mipPath = filePath[:-1] + 'a_mip';
elif str.endswith(filePath,'d'):
    mipPath = filePath[:-1] + 'c_mip';
viewer.open((dirData+mipPath + '.nd2'), plugin='napari-nikon-nd2', blending='additive')

# viewer.layers.remove(viewer.layers[len(viewer.layers)-1]) # remove transmitted detector image
viewer.layers.select_next();
eyeChannels = 0
if len(viewer.layers)==2+nMIP+nChannels: # no DAPI, GFP and RFP
   eyeChannels = 2
   viewer.layers[nMIP+nChannels].colormap = 'green'
   viewer.layers[nMIP+nChannels].name = 'eyeGreen'
   viewer.layers[nMIP+nChannels+1].colormap = 'magenta'
   viewer.layers[nMIP+nChannels+1].name = 'eyeRed'
elif len(viewer.layers)==3+nMIP+nChannels: # leave DIC
   eyeChannels = 3
   viewer.layers[nMIP+nChannels+0].colormap = 'green'
   viewer.layers[nMIP+nChannels+0].name = 'eyeGreen'
   viewer.layers[nMIP+nChannels+1].colormap = 'magenta'
   viewer.layers[nMIP+nChannels+1].name = 'eyeRed'
   viewer.layers[nMIP+nChannels+2].colormap = 'gray'
   viewer.layers[nMIP+nChannels+2].name = 'eyeBlue'
   viewer.layers[nMIP+nChannels+2].opacity = 0.33
elif len(viewer.layers)==4+nMIP+nChannels: #DIC + DAPI
   eyeChannels = 4
   viewer.layers[nMIP+nChannels+0].colormap = 'bop blue'
   viewer.layers[nMIP+nChannels+0].name = 'eyeBlue'
   viewer.layers[nMIP+nChannels+1].colormap = 'green'
   viewer.layers[nMIP+nChannels+1].name = 'eyeGreen'
   viewer.layers[nMIP+nChannels+2].colormap = 'magenta'
   viewer.layers[nMIP+nChannels+2].name = 'eyeRed'
   viewer.layers[nMIP+nChannels+3].colormap = 'gray'
   viewer.layers[nMIP+nChannels+3].name = 'eyeDIC'
   viewer.layers[nMIP+nChannels+3].opacity = 0.33
print(eyeChannels)
print('Loaded: ' + filePath)
eyeMarks = viewer.add_points(size=20, name = 'eyeMarks', symbol='cross', face_color='#ffffff80')
viewer.layers.select_next();

# Save eye landmarks
eyeMarksButton = ipywidgets.Button(description='Save eyeMarks', layout=ipywidgets.Layout(width='200px', height='100px'))
out = ipywidgets.Output()
def eyeMClick(_):
   with out:
      l = viewer.layers['eyeMarks']; l.save(dirOut + l.name + '.csv');
      print('eyeMarks layers saved for ' + filePath)
   # clear viewer
   viewer.layers.select_all()
   viewer.layers.remove_selected()
   eyeMarksButton.disabled = True # to prevent double clicking warnings. All sales are final!

eyeMarksButton.on_click(eyeMClick)

print('Make 6 landmark points then click button to save')

ipywidgets.VBox([eyeMarksButton,out])

### [Go back and run next one &uarr;](#zExtract)

***
## Segmentation using cellpose<a id='cellSeg'>∮</a>
> Define list of files to segment &rarr; Define cellpose parameters &rarr; Run cellpose in each image
***
[Back to Index](#Index)

### Running cellpose as batch analysis
Usually for:
- Rods: diameter = 20; flow_threshold = 0.5 and mask_threshold = 0.0
- Cones: diameter = 40; flow_threshold = 0.5 and mask_threshold = 0.0
- Horizontal cells: diameter = 100; flow_threshold = 0.6 and mask_threshold = 0.0

> Using a higher _flow_threshold_ (i.e. 0.6 instead of 0.4) will provide more ROIs

In [ ]:
# get file list again
gene = 'lhx1a'
print(*getFileList(dirData,gene,fileNameMatch,addDetails='list'), sep = "\n") #display the file list for copy and paste below

In [ ]:
# clear viewer
viewer.layers.select_all(); viewer.layers.remove_selected()

# collect file paths for wild-type
gene = 'wt'
filePaths_wt = [

         ]
# collect file paths for mutants
gene = 'lhx1a'
filePaths_F0 = [
   # '20230209_5dpf_m2G_s2C_lhx1aF0_L01_002',
   # '20230209_5dpf_m2G_s2C_lhx1aF0_L01_004',
   # '20230209_5dpf_m2G_s2C_lhx1aF0_L02_002',
   # '20230209_5dpf_m2G_s2C_lhx1aF0_L02_004',
   # '20230209_5dpf_m2G_s2C_lhx1aF0_L03_002',
   # '20230209_5dpf_m2G_s2C_lhx1aF0_L04_002',
   # '20230209_5dpf_m2G_s2C_lhx1aF0_L04_004',
   # '20230209_5dpf_m2G_s2C_lhx1aF0_L05_002',
   # '20230209_5dpf_m2G_s2C_lhx1aF0_L05_004',
   # '20230209_5dpf_m2G_s2C_lhx1aF0_L06_002',
   # '20230209_5dpf_m2G_s2C_lhx1aF0_L06_004',
   # '20230209_5dpf_m2G_s2C_lhx1aF0_L07_002',
   # '20230209_5dpf_m2G_s2C_lhx1aF0_L07_004',
   # '20230209_5dpf_m2G_s2C_lhx1aF0_L08_002',
   # '20230209_5dpf_m2G_s2C_lhx1aF0_L08_004',
   # '20230209_5dpf_m2G_s2C_lhx1aF0_L09_002',
   # '20230209_5dpf_m2G_s2C_lhx1aF0_L09_004',
   # '20230209_5dpf_m2G_s2C_lhx1aF0_L10_002',
   # '20230209_5dpf_m2G_s2C_lhx1aF0_L10_004',
   # '20230209_5dpf_m2G_s2C_lhx1aF0_L11_002',
   # '20230209_5dpf_m2G_s2C_lhx1aF0_L11_004',
   # '20230209_5dpf_m2G_s2C_lhx1aF0_L12_002',
   # '20230209_5dpf_m2G_s2C_lhx1aF0_L12_004',
   # '20230209_5dpf_m2G_s2C_lhx1aF0_L13_002',
   '20230209_5dpf_m2G_s2C_lhx1aF0_L13_004',
         ]


# define cellpose parameters
cpParams = {
    'model' : 'cyto2', # default is 'cyto' or 'cyto2'
    'net_avg' : True,
    'channels' : [0,0], #single channel without nucleus info
    'diameterR' : 25,
    'diameterC' : 40,
    'diameterH' : 100,
    'flow_threshold' : 0.5,
    'cellprob_threshold' : 0.0
}


# define model to use (e.g. 'cyto2')
model = models.Cellpose(gpu=False, model_type=cpParams['model'])

#### Run in loop to track progress and automatically save results
> In Jan 2022: ~4 minutes per 1024 x 1024 image (4 nets). 

In [ ]:
print('Starting analysis for wt:')
with tqdm(total=len(filePaths_wt)*len(layerNames), file=sys.stdout) as progBar:
   for fp in filePaths_wt:
      print('\t' + fp)
      for l in layerNames:
         print('\t\t ' + layerNames[l])
         startTime = time.time()
         imgPath = dirAnalysis + gene + '/' + fp + '/' + layerNames[l] + '_mip.tiff'
         img = cellpose.io.imread(imgPath)
         #     imgplot = plt.imshow(img)
         masks, flows, styles, diams = model.eval(img, 
                                                diameter=cpParams['diameterC'], channels=cpParams['channels'],
                                                do_3D=False, net_avg = cpParams['net_avg'], interp = True,
                                                flow_threshold = cpParams['flow_threshold'], cellprob_threshold = cpParams['cellprob_threshold'])
         cellpose.io.masks_flows_to_seg(img, masks, flows, diams, imgPath, cpParams['channels']) # save results
         endTime = time.time()
         print ('\t Time elapsed: {elapsedTime} s\n'.format(elapsedTime = int(endTime - startTime)))
         progBar.update(1)

print('Finished cellpose batch analysis for wt\n\n')

print('Starting analysis for '+ gene + ':')
with tqdm(total=len(filePaths_F0)*len(layerNames), file=sys.stdout) as progBar:
   for fp in filePaths_F0:
      print('\t' + fp)
      for l in layerNames:
         print('\t\t ' + l)
         startTime = time.time()
         imgPath = dirAnalysis + gene + '/' + fp + '/' + l + '_mip.tiff'
         img = cellpose.io.imread(imgPath)
         #     imgplot = plt.imshow(img)
         masks, flows, styles, diams = model.eval(img, 
                                                diameter=cpParams['diameterC'], channels=cpParams['channels'],
                                                do_3D=False, net_avg = cpParams['net_avg'], interp = True,
                                                flow_threshold = cpParams['flow_threshold'], cellprob_threshold = cpParams['cellprob_threshold'])
         cellpose.io.masks_flows_to_seg(img, masks, flows, diams, imgPath, cpParams['channels']) # save results
         endTime = time.time()
         print ('\t\t\t Time elapsed: {elapsedTime} s'.format(elapsedTime = int(endTime - startTime)))
         progBar.update(1)

print('Finished cellpose batch analysis for '+ gene)

***
## Manual correction of cellpose segmentation<a id='manualCuration'>∮</a>
> Copy file list from "Extract Layers" &rarr; Open a single file and manually fix ("_curate_") cellpose segmentation

WARNINGS:
- In tg[_mws2_:GFP], a subset of S cones are GFP+. If available, use tg[_sws2_:mCherry] to delete those labels from the M-cone segmentation
***
[Back to Index](#Index)

In [6]:
# Redefine layer names if necessary
# layerNames = ['M','S']
layerNames = ['M','S','N']

# clear viewer
viewer.layers.select_all(); viewer.layers.remove_selected()

#mutant files
filePath = '20230209_5dpf_m2G_s2C_lhx1aF0_L13_004'; gene = 'lhx1a';

# define subdirectory for analysis
dirOut = dirAnalysis + gene + '/' + filePath + '/'
print('Viewer cleared...')

# clear key binds
@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
    ...
@viewer.bind_key('b', overwrite=True)
def toggle_sel(viewer):
    ...
@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
    ...

# load mips
mipO = 0.65
viewer.open(dirOut + layerNames[0] + "_mip.tiff", plugin='napari', colormap = 'green', blending='additive', opacity=mipO);
viewer.open(dirOut + layerNames[1] + "_mip.tiff", plugin='napari', colormap = 'magenta', blending='additive', opacity=mipO);
viewer.open(dirOut + layerNames[2] + "_mip.tiff", plugin='napari', colormap = 'gray', blending='additive', opacity=mipO);

# load segmentation
segData = np.load(dirOut + layerNames[0] + "_mip" + "_seg.npy", allow_pickle=True).item()
viewer.add_labels(segData['masks'], name= layerNames[0] + '_seg',blending='additive');
viewer.layers[layerNames[0] + '_seg'].preserve_labels = True;

segData = np.load(dirOut + layerNames[1] + "_mip" + "_seg.npy", allow_pickle=True).item()
viewer.add_labels(segData['masks'], name= layerNames[1] + '_seg',blending='additive');
viewer.layers[layerNames[1] + '_seg'].preserve_labels = True;

viewer.layers[layerNames[0] + '_mip'].visible = False
viewer.layers[layerNames[2] + '_mip'].visible = False
viewer.layers[layerNames[0] + '_seg'].visible = False

print('Loaded  ' + filePath + ' !\n Now fix it and save it!')

#define useful keyboard shortcuts

@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
   lname = layerNames[1] + '_seg'
   tempd = viewer.layers[lname].data
   tempd[tempd == viewer.layers[lname].selected_label]=0
   viewer.layers[lname].data = tempd
   print('Cut!')
print('removeLabel on $0_seg ("Shift-X")'.format(layerNames[1]))

@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
   lname = layerNames[0] + '_seg'
   if (viewer.layers[lname].preserve_labels == True):
      viewer.layers[lname].preserve_labels = False
   elif (viewer.layers[lname].preserve_labels == False):
      viewer.layers[lname].preserve_labels = True
print('toggle $0_seg preserve_labels ("k")'.format(layerNames[1]))
        
@viewer.bind_key('b', overwrite=True)
def toggle_sel(viewer):
   lname = layerNames[1] + '_mip'
   if (viewer.layers[lname].visible == True):
      viewer.layers[lname].visible = False
   elif (viewer.layers[lname].visible == False):
      viewer.layers[lname].visible = True
print('toggle $0_mip visibility ("B")'.format(layerNames[0]))

@viewer.bind_key('n', overwrite=True)
def toggle_sel(viewer):
   lname = layerNames[0] + '_mip'
   if (viewer.layers[lname].visible == True):
      viewer.layers[lname].visible = False
   elif (viewer.layers[lname].visible == False):
      viewer.layers[lname].visible = True
print('toggle $0_mip visibility ("N")'.format(layerNames[2]))

@viewer.bind_key('+', overwrite=True)
def new_label(viewer):
   """Set the currently selected label to the largest used label plus one."""
   lname = layerNames[1] + '_seg'
   viewer.layers[lname].selected_label = viewer.layers[lname].data.max() + 1
print('Add new label in $0_seg visibility ("+"|"M")'.format(layerNames[1]))

Viewer cleared...
Loaded  20230209_5dpf_m2G_s2C_lhx1aF0_L13_004 !
 Now fix it and save it!
removeLabel on $0_seg ("Shift-X")
toggle $0_seg preserve_labels ("k")
toggle $0_mip visibility ("B")
toggle $0_mip visibility ("N")
Add new label in $0_seg visibility ("+"|"M")


#### resave first curated segmentation after napari-ing around

In [ ]:
baseName = 'R'
lname = baseName + '_seg'
l = viewer.layers[baseName + '_seg']; l.save(dOut + l.name + '_curated.tiff')
print('Done with ' + lname + ' for ' + fPath)

viewer.layers['U_mip'].visible = True
viewer.layers['U_seg'].visible = True
viewer.layers['U_seg'].contour = 0
viewer.layers['R_mip'].visible = False
viewer.layers['R_seg'].visible = False

@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
    lname = 'U_seg'
    tempd = viewer.layers[lname].data
    tempd[tempd == viewer.layers[lname].selected_label]=0
    viewer.layers[lname].data = tempd
    print('Cut!')

print('removeLabel on U_seg ("Shift-X")')

@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
    lname = 'U_seg'
    if (viewer.layers[lname].preserve_labels == True):
        viewer.layers[lname].preserve_labels = False
    elif (viewer.layers[lname].preserve_labels == False):
        viewer.layers[lname].preserve_labels = True
print('toggle U_seg preserve_labels ("k")')

@viewer.bind_key('b', overwrite=True)
def toggle_sel(viewer):
    lname = 'R_mip'
    if (viewer.layers[lname].visible == True):
        viewer.layers[lname].visible = False
    elif (viewer.layers[lname].visible == False):
        viewer.layers[lname].visible = True
print('toggle R_mip visibility ("B")')

        
@viewer.bind_key('n', overwrite=True)
def toggle_sel(viewer):
    lname = 'U_mip'
    if (viewer.layers[lname].visible == True):
        viewer.layers[lname].visible = False
    elif (viewer.layers[lname].visible == False):
        viewer.layers[lname].visible = True
print('toggle U_mip visibility ("N")')

#### resave second curated segmentation after napari-ing around

In [ ]:
baseName = 'U'
lname = baseName + '_seg'
l = viewer.layers[baseName + '_seg']; l.save(dOut + l.name + '_curated.tiff')
print('Done with ' + lname + ' for ' + fPath)

# # If NUCS mip revealed UV cones
baseName = 'U'
lname = baseName + '_missing'
l = viewer.layers['Points']; l.save(dOut + baseName + '_missing');
print("Image has {0} UV-cone nuclei".format(len(viewer.layers['Points'].data)))

### Reload curated segmentations after saving for final review

In [ ]:
# remove all except mips
# viewer.layers.remove(viewer.layers['Points'])
n_mips = 3;
if len(viewer.layers)>n_mips:
    for l in viewer.layers[n_mips:]:
        viewer.layers.remove(l)

viewer.open(dOut + 'U' + "_seg_curated.tiff", name='U_seg', plugin='builtins',blending='additive');
# viewer.layers['U_seg'].contour = 5
viewer.layers['U_mip'].visible = True
viewer.open(dOut + 'R' + "_seg_curated.tiff", name='R_seg', plugin='builtins',blending='additive');

print(fPath)
nR = len(np.unique(viewer.layers['R_seg'].data))-1
print("Image has {0} Rods".format(nR))
nU = len(np.unique(viewer.layers['U_seg'].data))-1
print("Image has {0} UV cones".format(nU))

### [Go back and run next one &uarr;](#manualCuration)

***
## Quantification<a id='quantification'></a>
***
[Back to Index](#Index)

### Read csv created during analysis and create bar plot

In [ ]:
#gene Colors
zfC = {
    'R'  : '#7d7d7d',
    'U' : '#B73AB9',
    'S' : '#4364F6',
    'M' : '#59CB3B',
    'L' : '#CE2A22',
}

zfG = {
    'wt' : '#000000',
    'tbx2a' : '#ab266b',
    'tbx2b' : '#421f8e',
    'foxq2' : '#001dd6',
    'nr2e3' : '#7d7d7d',
}

zfGm = {
    'wt' : 'o',
    'tbx2a' : 'P',
    'tbx2b' : 'X',
    'nr2e3' : '+',
    
}

prLabel = {
    'R'  : 'Rods',
    'U' : 'UV',
    'S' : 'S',
    'M' : 'M',
    'L' : 'L',
}


def formatFigureMain(figH, axH, plotH):
#     font_path = 'C:/Users/pataklk/Documents/Frag_Analysis_Code/Avenir.ttc'
    font_path = '/System/Library/Fonts/Avenir.ttc'
    fontTicks = font_manager.FontProperties(fname=font_path, size=30) # was 18
    fontLabels = font_manager.FontProperties(fname=font_path, size=36) # was 22
    fontTitle = font_manager.FontProperties(fname=font_path, size=30) # was 28 
    axH.set_xscale('linear')
    axH.spines['top'].set_visible(False)
    axH.spines['right'].set_visible(False)
    
    for label in (axH.get_xticklabels() + axH.get_yticklabels()):
        label.set_fontproperties(fontTicks)
    axH.set_xlabel(axH.get_xlabel(), fontproperties = fontTicks)
    axH.set_ylabel(axH.get_ylabel(), fontproperties = fontTicks)
    return fontLabels

def formatFigure(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
#     axH.set_xlabel('wt vs. cr', fontproperties=fontLabels)
    axH.set_ylabel('cells per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)

def formatFigureRvU(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
    axH.set_xlabel('Rods per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.set_ylabel('UV cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)
    
def formatFigureMvS(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
    axH.set_xlabel('M cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.set_ylabel('S cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)

    
def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import colorsys
    try:
        c = matplotlib.colors.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*matplotlib.colors.to_rgb(c))
    return matplotlib.colors.rgb2hex(colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2]))

def estimateJitter(dataArray):
    """ creates random jitter scaled by local density of points"""
    from scipy.stats import gaussian_kde
    kde = gaussian_kde(dataArray)
    density = kde(dataArray)
    jitter = np.random.randn(len(dataArray))*density
    return jitter
pBGMenu

In [ ]:
#reapply plotting style
applyPlotStyle(pBGMenu.value)

In [ ]:
# Created csv manually during analysis
dPath = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/xOGaCT/"
fName = "CRnr2e3F0s_xOGaCT_Counts.csv"

dPath = "/Users/angueyraaristjm/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/zf/Analysis/CRnr2e3F0s/"
# dPath = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/m2Gs2C/"
fName = "CRnr2e3F0s_combinedCounts.csv"

df = pd.read_csv(dPath + fName)
df

In [ ]:
geneList = ['wt','nr2e3']
nGenes = np.size(geneList)
photoreceptors = ['R','U','S','M','L']
# photoreceptors = ['S','M']

plotname = ''
fH, axH = plt.subplots(figsize= [12,8])

barW = nGenes+2; # bar width
barD = nGenes+1; # bar distance whitin 1 photoreceptor subtype
barP = -nGenes+1; # position in x-axis
barStep = 0.9; # distance between photoreceptor groups
barPos = []; #array to save bar positions


j=barP;
for gene in geneList:
    i=0;
    j=j+1;
    for pr in photoreceptors:
        i = i+barStep;
        countData = df[(df['genotype']==gene) & (df['excludeFlag']==False)][pr]
        jitter = np.random.randn(len(countData))*0.03
#         textindent = 1/(2.5*barD);
        textindent = 0.10;
        pH = plt.bar([i+j/barD], np.mean(countData), width=1/barW, color=lighten_color(zfC[pr],1), linewidth = 2, edgecolor = lighten_color(zfC[pr],1)); #color=zfC[pr], 
#         pH = plt.bar([i+j/barD], np.mean(countData), yerr = sem(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr][~np.isnan(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr])], ddof = 0), align='center', ecolor='black', capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
#         pH = plt.bar([i+j/barD], np.mean(countData), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr][~np.isnan(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr])], ddof = 0), align='center', ecolor='black', capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
       # pH = plt.bar([i+j/barD], np.mean(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), ecolor=lighten_color(zfC[pr],1.2), capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
        #pH = plt.errorbar([i+j/barD], np.mean(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), ecolor=lighten_color(zfC[pr],1.1), elinewidth=2, capsize=5, capthick=2, barsabove= True)
#         pH = plt.scatter(np.ones(len(countData))*[i+j/barD]+jitter, countData, color=lighten_color(zfC[pr],2/3), zorder=2, marker = zfGm[gene], edgecolor='#ffffff', linewidth=0.5, alpha = 0.4);
        pH = plt.errorbar(i+j/barD, np.mean(countData), yerr = [[0],[np.std(countData)]], ecolor=lighten_color(zfC[pr],1), elinewidth=3, capsize=8, capthick=3, zorder=9)
        # plt.text((i+j/barD)-textindent, 3, prLabel[pr], font_properties=font_prop, fontsize=20, ha='left', alpha=0.6)
        pH = plt.scatter(np.ones(len(countData))*[i+j/barD]+jitter, countData, color='#ffffff', zorder=8, marker = 'o', s=50, edgecolor='#ffffff', linewidth=0.5, alpha = .6);
        barPos = np.append(barPos,(i+j/barD))

formatFigure(fH, axH, pH)
axH.set_xticks(np.sort(barPos));

axH.set_xticklabels(geneList * len(photoreceptors));
# axH.set_ylim([0,500]); # this was 400

# savePath = 'C:/Users/pataklk/OneDrive - National Institutes of Health/zf/F0_Analysis/CRfoxq2F0/'
# savePath = "/Users/angueyraaristjm/OneDrive - National Institutes of Health/zf/F0_Analysis/CRfoxq2F0/"
# savePath = "/Users/angueyraaristjm/Documents/LiLab/Presentations/revealjs/resources/20211008_UCLA/"
# plt.savefig(savePath + "Counts_foxq2.png", transparent=True, format="png", bbox_inches = "tight")
# plt.savefig(savePath + "Counts_foxq2_B.svg", transparent=True, format="svg", bbox_inches = "tight")

### Stats

### Read csv created during analysis and create bar plot

In [ ]:
geneList = ['wt','nr2e3']
photoreceptors = ['R','U','S','M','L']

print('{0} vs. {1}:'.format(geneList[0],geneList[1]))

for pr in photoreceptors:
    # get counts for each photoreceptor subtype and exclude NaNs
    wtCount = df[(df['genotype']==(geneList[0]))&(df['excludeFlag']==False)][pr]
    wtCount = wtCount[~np.isnan(wtCount)]
    
    crCount = df[(df['genotype']==(geneList[1]))&(df['excludeFlag']==False)][pr]
    crCount = crCount[~np.isnan(crCount)]
    
    u, p = mannwhitneyu(wtCount, crCount)
    print('\t{0}:'.format(pr))
    print('\t\tU = {0:.3f}, p = {1:.5f}, nEyes: wt = {2:.0f}; cr = {3:.0f}'.format(u,p,len(wtCount),len(crCount)))


### Compile counts by looping through all files

In [ ]:
def cellCounter(viewer,dAnalysis,gene,fPath,photoLabel):
    import os.path
    # clear viewer
    for l in viewer.layers:
        viewer.layers.remove(l)
    viewer.layers.select_all()
    viewer.layers.remove_selected()
    dOut = dAnalysis + gene + '/' + fPath + '/'
    # load segmentation
    segPath = dOut + photoLabel + "_seg_curated.tiff"
    if os.path.isfile(segPath):
        viewer.open(segPath, name='Seg', plugin='builtins', blending='additive');
        nCells = np.unique(viewer.layers['Seg'].data).shape[0]
    else:
        nCells = float("nan")
    print(fPath + ': nR = ' + str(nCells)) 
#     np.savez(dOut + 'quantificationGFP.npz',
#              lcones=lcones,
#              lRFP=lRFP,
#              lGFP=lGFP,
#              lRFPsd=lRFPsd,
#              lGFPsd=lGFPsd,
#              lRtile=lRtile,
#              lGtile=lGtile)
#     results = zip(['lcones','lRFP','lGFP','lRFPsd','lGFPsd','lRtile','lGtile'],
#                  [lcones,lRFP,lGFP,lRFPsd,lGFPsd,lRtile,lGtile])
    
    return nCells

In [2]:
import os.path
import time, sys, warnings, re
from tqdm.notebook import tqdm, trange
import importlib
import ipywidgets
from IPython.display import display
# lab brews
from plotParams import *
from lloyd import *

In [8]:
# define where all the imaging data is stor\d
dirData = 'D:/Genotyping/20220128_tbx2F3_nr2e3F0/tbx2a_inx_nr2e3/'
# define where all the analysis files should be stored
# subdirectories for each gene and transgenic lines need to be manually made first
dirAnalysis = "/Users/juanangueyra/Library/CloudStorage/GoogleDrive-angueyra@umd.edu/Shared drives/vldImaging/Analysis/F0lhx1a/m2Gs2C/"

In [9]:
gene = 'fsa' # usually 'wt' or gene of interest (e.g. 'tbx2a', 'lhx1a')
fileNameMatch = [''] # characters in filenames that define zoom-in files

# get a list with all the relevant files
print(*getFileList(dirData,gene,fileNameMatch,addDetails=''), sep = "\n") #display the file list for copy and paste below

tbx2ainx_nr2e3inj_nr2e3_01_H08.fsa
tbx2ainx_nr2e3inj_nr2e3_02_G08.fsa
tbx2ainx_nr2e3inj_nr2e3_03_F08.fsa
tbx2ainx_nr2e3inj_nr2e3_04_E08.fsa
tbx2ainx_nr2e3inj_nr2e3_05_D08.fsa
tbx2ainx_nr2e3inj_nr2e3_06_C08.fsa
tbx2ainx_nr2e3inj_nr2e3_07_B08.fsa
tbx2ainx_nr2e3inj_nr2e3_08_A08.fsa
tbx2ainx_nr2e3inj_nr2e3_09_H09.fsa
tbx2ainx_nr2e3inj_nr2e3_10_G09.fsa
tbx2ainx_nr2e3inj_nr2e3_11_F09.fsa
tbx2ainx_nr2e3inj_nr2e3_12_E09.fsa
tbx2ainx_nr2e3inj_nr2e3_13_D09.fsa
tbx2ainx_nr2e3inj_nr2e3_14_C09.fsa
tbx2ainx_nr2e3inj_nr2e3_15_B09.fsa
tbx2ainx_nr2e3inj_nr2e3_16_A09.fsa
tbx2ainx_nr2e3inj_nr2e3_17_H10.fsa
tbx2ainx_nr2e3inj_nr2e3_18_G10.fsa
tbx2ainx_nr2e3inj_nr2e3_19_F10.fsa
tbx2ainx_nr2e3inj_nr2e3_20_E10.fsa
tbx2ainx_nr2e3inj_nr2e3_21_D10.fsa
tbx2ainx_nr2e3inj_nr2e3_22_C10.fsa
tbx2ainx_nr2e3inj_nr2e3_23_B10.fsa
tbx2ainx_nr2e3inj_nr2e3_24_A10.fsa
tbx2ainx_nr2e3inj_nr2e3_25_D07.fsa
tbx2ainx_nr2e3inj_nr2e3_26_C07.fsa
tbx2ainx_nr2e3inj_nr2e3_27_B07.fsa
tbx2ainx_nr2e3inj_nr2e3_28_A07.fsa
tbx2ainx_uninj_nr2e3

In [6]:
96-57

39

In [7]:
55-16

39